In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor



# 处理数据

PyTorch 提供了两个用于 处理数据的原语<https://pytorch.org/docs/stable/data.html>: torch.utils.data.DataLoader 和 torch.utils.data.Dataset。`Dataset`存储样本及其对应的标签，而`DataLoader`则在`Dataset`外部封装一层，变为可迭代对象。


PyTorch 提供了特定领域的库，例如 [TorchText](https://pytorch.org/text/stable/index.html)， [TorchVision](https://pytorch.org/vision/stable/index.html)，和 [TorchAudio](https://pytorch.org/audio/stable/index.html)， 所有这些库都包含了对应数据集。在本教程中，我们将使用 TorchVision 数据集。

In [3]:
# `torchvision.datasets` 模块包含许多现实世界视觉数据`Dataset`，例如 CIFAR、COCO
# ([数据集列表](https://pytorch.org/vision/stable/datasets.html))。
# 在本教程中，我们使用 `FashionMNIST`数据集。
# 每个TorchVision `Dataset`包括两个参数：`transform` 和 `target_transform`，分别用于修改样本数据和标签。

# Download training data from open datesets
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)


# Download test data from open datasets

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()

)


我们将 Dataset 作为参数传递给 DataLoader，在数据集上封装了一个可迭代对象， 支持自动批处理、采样、打乱和多进程数据加载。这里我们定义一个批处理大小为 64， 即 dataloader 每批将返回大小为 64 的特征数据和标签

In [4]:
batch_size = 64

# Create data loaders
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}]")
    print(f"Shape of y {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])]
Shape of y torch.Size([64]) torch.int64


# 创建模型
要在 PyTorch 中定义一个神经网络，我们需要创建一个继承自 nn.Module<https://pytorch.org/docs/stable/generated/torch.nn.Module.html> 的类。 我们在 `__init__ 函数中定义网络的层，并在 forward 函数中指定数据如何经过网络。为了加速神经网络中的运算， 我们将其移到 GPU 或 MPS（如果可用）上。

In [9]:
input = torch.randn(32, 1, 5, 5)
m = nn.Flatten()
print(input.shape)
out = m(input)
print(out.shape)
m1 = nn.Flatten(0, 2)
out1 = m1(input)
print(out1.shape)

torch.Size([32, 1, 5, 5])
torch.Size([32, 25])
torch.Size([160, 5])


In [10]:
# Get cpu, gpu or mps device for training
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")



class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits



model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# Optimizing the Model Parameters

To train a model, we need a `loss` function and `optimizer`

In [11]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

在单个训练循环中，模型对训练数据集（分批输入）进行预测，并通过反向传播预测误差来调整模型的参数。

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # output training process
        if batch % 100 ==0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:5d}/{size:>5d}]" )


We also check the model's performance against the test dataset to ensure it is learning.

我们还需检查模型在测试数据集上的效果，以确保它在持续学习

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

通过多次迭代（epochs）进行训练。在每个迭代过程中，模型通过对参数的学习以提高预测准确性。 我们在每个 epoch 打印模型的准确率和损失；我们希望看到随着每个 epoch 训练，模型预测准确率不断提高， 损失逐渐减少。

In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1} \n" + "-" * 10)
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1 
----------
loss: 2.173643  [   64/60000]
loss: 2.168273  [ 6464/60000]
loss: 2.108286  [12864/60000]
loss: 2.129576  [19264/60000]
loss: 2.069617  [25664/60000]
loss: 2.012483  [32064/60000]
loss: 2.040752  [38464/60000]
loss: 1.956508  [44864/60000]
loss: 1.967414  [51264/60000]
loss: 1.892366  [57664/60000]
Test Error: 
 Accuracy: 46.6%, Avg loss: 1.892198 

Epoch 2 
----------
loss: 1.927734  [   64/60000]
loss: 1.899978  [ 6464/60000]
loss: 1.774837  [12864/60000]
loss: 1.820385  [19264/60000]
loss: 1.711339  [25664/60000]
loss: 1.659905  [32064/60000]
loss: 1.684280  [38464/60000]
loss: 1.576700  [44864/60000]
loss: 1.608145  [51264/60000]
loss: 1.504781  [57664/60000]
Test Error: 
 Accuracy: 57.9%, Avg loss: 1.519496 

Epoch 3 
----------
loss: 1.591242  [   64/60000]
loss: 1.556443  [ 6464/60000]
loss: 1.398852  [12864/60000]
loss: 1.477522  [19264/60000]
loss: 1.367298  [25664/60000]
loss: 1.356968  [32064/60000]
loss: 1.376260  [38464/60000]
loss: 1.290712  [44864/600

# Saving Models

A Common way to save model is to serialize the internal state dictionary (containing the model parameters)

In [17]:
torch.save(model.state_dict(), 'model.pth')
print("Save PyTorch Model State to model.pth")

Save PyTorch Model State to model.pth


# Loading Models

The process for loading model includes recreating model structure and loading the state dictionary into it.

In [18]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('model.pth'))


<All keys matched successfully>

This model can now be used to make prediction

In [20]:
classes = [
    "T-shirt/top",
    "Trouser", # 长裤
    "Pullover", # 套衫
    "Dress",
    "Coat",
    "Sandal", # 凉鞋
    "Shirt",
    "Sneaker", # 运动鞋
    "Bag",
    "Ankle boot", # 短靴
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
